In [7]:
import cv2
import mediapipe as mp
import numpy as np
from math import sqrt

# Initialize MediaPipe Pose solution
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, enable_segmentation=False, min_detection_confidence=0.5)

# Initialize MediaPipe drawing
mp_drawing = mp.solutions.drawing_utils

# Initialize video capture
cap = cv2.VideoCapture(0)

# Function to calculate Euclidean distance between two landmarks
def calculate_distance(landmark1, landmark2):
    return sqrt((landmark1.x - landmark2.x) ** 2 + (landmark1.y - landmark2.y) ** 2)

# Function to check for significant change in distances
def has_significant_change(new_distances, prev_distances, threshold=0.01):
    for part in new_distances:
        if new_distances[part] is not None and abs(new_distances[part] - prev_distances.get(part, 0)) > threshold:
            return True
    return False

# Variables for squat counting
squat_count = 0
squat_started = False
squat_threshold = 0.15  # Threshold for recognizing a squat

# Open a file to write the distances
with open("squat_landmark_distances.txt", "w") as file:
    prev_distances = {}
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the image and find the pose landmarks
        results = pose.process(image)

        # Draw the pose annotation on the image
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
            # Get landmark positions
            landmarks = results.pose_landmarks.landmark

            # Define the landmarks of interest
            landmarks_of_interest = {
                'left_ankle': mp_pose.PoseLandmark.LEFT_ANKLE,
                'left_knee': mp_pose.PoseLandmark.LEFT_KNEE,
                'left_hip': mp_pose.PoseLandmark.LEFT_HIP,
                'left_elbow': mp_pose.PoseLandmark.LEFT_ELBOW,
                'left_eye': mp_pose.PoseLandmark.LEFT_EYE
            }

            distances = {
                'left_ankle_to_left_knee': None,
                'left_ankle_to_left_hip': None,
                'left_ankle_to_left_elbow': None,
                'left_ankle_to_left_eye': None
            }
            
            # Calculate distances from left ankle to other parts
            if landmarks[landmarks_of_interest['left_ankle'].value].visibility > 0.5:
                left_ankle = landmarks[landmarks_of_interest['left_ankle'].value]
                for part, landmark in landmarks_of_interest.items():
                    if 'left_ankle' not in part and landmarks[landmark.value].visibility > 0.5:
                        distances[f'left_ankle_to_{part}'] = calculate_distance(left_ankle, landmarks[landmark.value])

            # Record the distances if there's a significant change
            if has_significant_change(distances, prev_distances):
                distances_list = [str(distances[key]) if distances[key] is not None else '' for key in distances]
                file.write(','.join(distances_list) + '\n')
                prev_distances = distances

            # Detect squats based on the vertical distance of the left knee to the left hip
            if landmarks[landmarks_of_interest['left_knee'].value].visibility > 0.5 and landmarks[landmarks_of_interest['left_hip'].value].visibility > 0.5:
                left_knee = landmarks[landmarks_of_interest['left_knee'].value]
                left_hip = landmarks[landmarks_of_interest['left_hip'].value]
                vertical_distance = left_hip.y - left_knee.y

                if vertical_distance > squat_threshold and not squat_started:
                    squat_started = True
                elif vertical_distance < squat_threshold and squat_started:
                    squat_started = False
                    squat_count += 1

            # Display the distances on the image
            y_offset = 30
            for key, distance in distances.items():
                cv2.putText(image, f"{key}: {distance:.4f}" if distance is not None else f"{key}: ", 
                            (10, y_offset), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                y_offset += 20

            # Display the squat count on the image
            cv2.putText(image, f"Squat Count: {squat_count}", 
                        (10, y_offset), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Display the image
        cv2.imshow('Squat Analysis', image)

        # Break the loop on 'q' key press
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release resources
cap.release()
cv2.destroyAllWindows()
